In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2  
import pyterrier as pt
import os
import numpy as np
import gc

print("Done")

Done


In [2]:

# --- PyTerrier Initialisierung ---
if not pt.started():
    pt.java.init()
print("Done")

/tmp/ipykernel_517996/1499247338.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [3]:
# --- Datenbankverbindung ---
DATABASE = "longeval-web"
USER = "dis18"
HOST = "db"
PORT = "5432"
PASSWORD = os.getenv("DB_PASSWORD", "dis182425")  

try:
    engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
    with engine.begin() as connection:
        print("Datenbankverbindung erfolgreich.")
except Exception as e:
    print(f"Fehler bei DB-Verbindung: {e}")
    exit()
print("Done")

Datenbankverbindung erfolgreich.
Done


In [4]:
import pandas as pd
from sqlalchemy import create_engine

DATABASE = "longeval-web"
USER = "dis18"
HOST = "db"
PORT = "5432"
PASSWORD = "dis182425"

engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

In [5]:

# --- SQL Queries ---
query_topic = """
SELECT 
    "Topic"."queryid" AS quer_id,    
    "Topic"."text_fr" AS query,
    "sub_collection" 
FROM "Topic"
WHERE "sub_collection" = '2023-08';
"""
#('2023-03')
query_qrel = """
SELECT DISTINCT
    "Qrel"."relevance" AS relevance,
    "queryid" as quer_id,
    "docid" as doc_id
    
from "Qrel"
WHERE "sub_collection" = '2023-08';


"""

query_document = """
SELECT 
    "Document"."url" AS url,
    "docid" as doc_id
from "Document"
WHERE "sub_collection" IN ('2023-08');
"""

query_vorkommen = """
WITH doc_counts AS (
    SELECT
        "docid",
        COUNT(*) AS "Anzahl_Vorkommen"
    FROM "Document"
    WHERE "sub_collection" IN (
        '2022-06', '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
        '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07',
        '2023-08'
    )
    GROUP BY "docid"
)
SELECT
    "docid",
    "Anzahl_Vorkommen"
FROM doc_counts;
"""


df_topic = pd.read_sql(query_topic, con=engine)
df_qrel = pd.read_sql(query_qrel, con=engine)
df_document = pd.read_sql(query_document, con=engine)
df_vorkommen = pd.read_sql(query_vorkommen, con=engine)

print(df_topic)
print(df_qrel)
print(df_document)
print(df_vorkommen)
print("Done")
# Anzahl eindeutiger Queries in den Qrels
num_qrels = df_qrel['quer_id'].nunique()
print(f"Einzigartige quer_id in df_qrel: {num_qrels}")



      quer_id                                 query sub_collection
0          67                      amygdale cerveau        2023-08
1           3                  1ere guerre mondiale        2023-08
2           8              4 mariages 1 enterrement        2023-08
3          12                          abri voiture        2023-08
4          18              acheter voiture occasion        2023-08
...       ...                                   ...            ...
12933   75421         domaine de chaumont sur loire        2023-08
12934   75422  restaurant l ile issy les moulineaux        2023-08
12935   75423                      cateau cambresis        2023-08
12936   75424                    filet anti oiseaux        2023-08
12937   75425                     petit chauffe eau        2023-08

[12938 rows x 3 columns]
Empty DataFrame
Columns: [relevance, quer_id, doc_id]
Index: []
                                                       url      doc_id
0        https://www.fnutopia.com.a

In [6]:
# --- BM25 Daten laden ---

bm25_file = "datasets/LongEval-Web/runs/longeval-web-fr-2023-08-BM25.gz"
bm_df = pt.io.read_results(bm25_file)
print(bm_df)
num_bm = bm_df['qid'].nunique()
print(f"Einzigartige quer_id in bm_df: {num_bm}")

print('Done')




            qid    docno  rank      score       name
0             3   684186     0  25.872507  pyterrier
1             3  2911357     1  25.872507  pyterrier
2             3  1686452     2  25.678216  pyterrier
3             3   430968     3  25.430486  pyterrier
4             3   658038     4  25.393302  pyterrier
...         ...      ...   ...        ...        ...
12583460  75425  1737432   995  18.909800  pyterrier
12583461  75425  1739913   996  18.909800  pyterrier
12583462  75425  3088634   997  18.909800  pyterrier
12583463  75425  2983643   998  18.909800  pyterrier
12583464  75425  2981689   999  18.909800  pyterrier

[12583465 rows x 5 columns]
Einzigartige quer_id in bm_df: 12865
Done


In [7]:
# --- Hilfsfunktion zur Speicheroptimierung bm_df ---
def optimize_df_types(df: pd.DataFrame) -> pd.DataFrame:
    """Optimizes DataFrame types in-place for memory."""
    for col in df.columns:
        col_type = df[col].dtype
        try:
            if col_type == 'float64':
                df[col] = df[col].astype(np.float32)
            elif col_type == 'int64':
                df[col] = pd.to_numeric(df[col], downcast='integer')
            elif col_type == 'object' and df[col].nunique() / len(df) < 0.5:
                 df[col] = df[col].astype('category')
        except Exception as e:
             # Optional: minimal error print if needed
             # print(f"Skipping {col}: {e}")
             pass # Ignore errors to keep it short
    return df

list_of_dfs = [bm_df]
optimized_list = [optimize_df_types(df) for df in list_of_dfs]
print("Done")

Done


In [8]:
# --- Hilfsfunktion zur Speicheroptimierung ---
def optimize_df_types(df: pd.DataFrame) -> pd.DataFrame:
    """Optimizes DataFrame types in-place for memory."""
    for col in df.columns:
        col_type = df[col].dtype
        try:
            if col_type == 'float64':
                df[col] = df[col].astype(np.float32)
            elif col_type == 'int64':
                df[col] = pd.to_numeric(df[col], downcast='integer')
            elif col_type == 'object' and df[col].nunique() / len(df) < 0.5:
                 df[col] = df[col].astype('category')
        except Exception as e:
             # Optional: minimal error print if needed
             # print(f"Skipping {col}: {e}")
             pass # Ignore errors to keep it short
    return df

list_of_dfs = [df_topic, df_qrel, df_document, df_vorkommen]
optimized_list = [optimize_df_types(df) for df in list_of_dfs]

print("Done")

Done


In [9]:

# und die Spalte mit den IDs heißt 'docid':

# Entferne das Präfix "doc" aus der 'docid'-Spalte
df_vorkommen['docid'] = df_vorkommen['docid'].str.replace(r'^doc', '', regex=True)



# Kontrolle
print(df_vorkommen)


          docid  Anzahl_Vorkommen
0             1                 1
1            10                14
2           100                 4
3          1000                14
4         10000                 2
...         ...               ...
3550129  999995                 2
3550130  999996                 6
3550131  999997                 6
3550132  999998                 4
3550133  999999                 3

[3550134 rows x 2 columns]


In [10]:
# Stelle sicher, dass beide Spalten vom selben Typ sind
bm_df['qid'] = bm_df['qid'].astype(str)
df_topic['quer_id'] = df_topic['quer_id'].astype(str)

print("Einzigartige qid in bm_df:", bm_df['qid'].nunique())


Einzigartige qid in bm_df: 12865


In [11]:
# Anzahl eindeutiger Queries in den Qrels
num_qrels = df_qrel['quer_id'].nunique()
print(f"Einzigartige quer_id in df_qrel: {num_qrels}")

# Anzahl eindeutiger Queries im BM25-Run
num_bm = bm_df['qid'].nunique()
print(f"Einzigartige qid in bm_df: {num_bm}")

Einzigartige quer_id in df_qrel: 0
Einzigartige qid in bm_df: 12865


In [12]:
Time_Tags = pd.read_csv('Expanded/Topics_With_TimeTags.csv')

# Nur Einträge mit '2023-01' in der Spalte 'sub_collection'
Time_Tags_202301 = Time_Tags[Time_Tags['sub_collection'] == '2023-08'].copy()
print(Time_Tags_202301)
print(bm_df)
unique_queryids = Time_Tags_202301['queryid'].nunique()

print(f"Anzahl einzigartiger queryid in sub_collection 2023-01: {unique_queryids}")

                                          id  queryid  text_en  \
140953  2d94cea6-46d6-4bd1-9ffa-b7438e9606ba        3      NaN   
140954  69ebeb31-3145-4e87-bad6-08eaa9c98aa8        8      NaN   
140955  a171dea6-6bb0-4bd4-8ace-6a5d26acc29c       12      NaN   
140956  38f184da-fe08-4b2f-87a9-365d91dca836       18      NaN   
140957  986b4402-e516-4868-bf61-67e82a277f61       19      NaN   
...                                      ...      ...      ...   
153886  4a862cb1-1e72-4aac-a9da-d327eb050ebb    75421      NaN   
153887  53700ef9-a867-45bb-809c-967973f9e68a    75422      NaN   
153888  2d667e52-bbfd-486d-8647-19a2d40f1251    75423      NaN   
153889  97e0a5eb-c4ca-4123-a73a-ab116afa58b4    75424      NaN   
153890  e771d467-7848-4692-810a-220efc4fc4ee    75425      NaN   

                                       query sub_collection split  tag  
140953                  1ere guerre mondiale        2023-08  test  0.2  
140954              4 mariages 1 enterrement        2023-08  

In [13]:
bm_df['qid'] = bm_df['qid'].astype(str)
Time_Tags_202301['queryid'] = Time_Tags_202301['queryid'].astype(str)

Final_df = pd.merge(
    bm_df,
    Time_Tags_202301,
    left_on="qid",
    right_on="queryid",
    how="inner"
)
print(Final_df)
# Ausgabe von Final_df sortiert nach der Spalte 'tag' (absteigend)
print(Final_df.sort_values(by='tag', ascending=False))


            qid    docno  rank      score       name  \
0             3   684186     0  25.872507  pyterrier   
1             3  2911357     1  25.872507  pyterrier   
2             3  1686452     2  25.678217  pyterrier   
3             3   430968     3  25.430487  pyterrier   
4             3   658038     4  25.393303  pyterrier   
...         ...      ...   ...        ...        ...   
12583460  75425  1737432   995  18.909800  pyterrier   
12583461  75425  1739913   996  18.909800  pyterrier   
12583462  75425  3088634   997  18.909800  pyterrier   
12583463  75425  2983643   998  18.909800  pyterrier   
12583464  75425  2981689   999  18.909800  pyterrier   

                                            id queryid  text_en  \
0         2d94cea6-46d6-4bd1-9ffa-b7438e9606ba       3      NaN   
1         2d94cea6-46d6-4bd1-9ffa-b7438e9606ba       3      NaN   
2         2d94cea6-46d6-4bd1-9ffa-b7438e9606ba       3      NaN   
3         2d94cea6-46d6-4bd1-9ffa-b7438e9606ba       3     

In [14]:
import pandas as pd

# 1. Spalte in df_vorkommen umbenennen
df_vorkommen_renamed = df_vorkommen.rename(columns={'docid': 'docno'})

# 2. Merge und direkt in Final_df zurückschreiben
Final_df = pd.merge(
    Final_df,
    df_vorkommen_renamed,
    on='docno',
    how='left'   # behält alle Zeilen aus Final_df bei
)

# 3. Kontrolle
# Ausgabe von Final_df sortiert nach der Spalte 'Anzahl_Vorkommen' (absteigend)
print(Final_df.sort_values(by='Anzahl_Vorkommen', ascending=True))



            qid    docno  rank      score       name  \
6732125   53061  3546151   249  15.098617  pyterrier   
7066092   56276  3542556     2  28.186865  pyterrier   
1963181    4087  3544074   345  18.575901  pyterrier   
7065864   56261  3549959   774   7.569582  pyterrier   
7065833   56261  3543770   743   7.578224  pyterrier   
...         ...      ...   ...        ...        ...   
9947279   69984    34794   333  13.421473  pyterrier   
4401389   35765  3173607   409  10.427013  pyterrier   
8169903   66546  3173607   367   7.783942  pyterrier   
10755497  70930  3173607   407  10.097882  pyterrier   
10885487  71074  3173607   706  14.504642  pyterrier   

                                            id queryid  text_en  \
6732125   c821bbf2-a515-4f22-8e66-fce774a6c574   53061      NaN   
7066092   214f10f6-f879-4bd5-9bad-54f99a22cd7a   56276      NaN   
1963181   56639cf9-f126-420a-94fc-4f56d1c34fe5    4087      NaN   
7065864   873bfb1c-9077-4a25-aabc-feeb1c246c85   56261     

In [15]:
# Zeige Datentypen aller Spalten in einem DataFrame, z.B. Final_df
print(Final_df.dtypes)

# Alternativ: ausführliche Übersicht mit info()
Final_df.info()


qid                   object
docno                 object
rank                   int16
score                float32
name                category
id                    object
queryid               object
text_en              float64
query                 object
sub_collection        object
split                 object
tag                  float64
Anzahl_Vorkommen       int16
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12583465 entries, 0 to 12583464
Data columns (total 13 columns):
 #   Column            Dtype   
---  ------            -----   
 0   qid               object  
 1   docno             object  
 2   rank              int16   
 3   score             float32 
 4   name              category
 5   id                object  
 6   queryid           object  
 7   text_en           float64 
 8   query             object  
 9   sub_collection    object  
 10  split             object  
 11  tag               float64 
 12  Anzahl_Vorkommen  int16   
dtypes: catego

In [16]:
# 1) Cappe 'Anzahl_Vorkommen' auf maximal 15(wegen extremer ausreißer, für die Normalisierung)
Final_df['Anzahl_Vorkommen'] = Final_df['Anzahl_Vorkommen'].clip(upper=15)

# 2) Recency-Feature neu berechnen
import numpy as np
Final_df['recency'] = 1 / (1 + np.log1p(Final_df['Anzahl_Vorkommen']))

# 3) Additiver, getaggter Bonus wie gehabt
lam = 1.0
Final_df['rerank_score'] = Final_df['score'] + lam * Final_df['tag'] * Final_df['recency']

# 4) Alte Spalten aufräumen
Final_df = Final_df.drop(columns=['score', 'recency'])

# 5) Optional sortieren
Final_df = Final_df.sort_values(['qid','rerank_score'], ascending=[True,False])

print(Final_df.head())




       qid    docno  rank       name                                    id  \
45072  100  1694624     0  pyterrier  1f263fef-7237-4e9a-ba6b-4c42813439d1   
45073  100  2971472     1  pyterrier  1f263fef-7237-4e9a-ba6b-4c42813439d1   
45075  100  2945156     3  pyterrier  1f263fef-7237-4e9a-ba6b-4c42813439d1   
45076  100  3071601     4  pyterrier  1f263fef-7237-4e9a-ba6b-4c42813439d1   
45074  100   143632     2  pyterrier  1f263fef-7237-4e9a-ba6b-4c42813439d1   

      queryid  text_en              query sub_collection split  tag  \
45072     100      NaN  appli pole emploi        2023-08  test  0.2   
45073     100      NaN  appli pole emploi        2023-08  test  0.2   
45075     100      NaN  appli pole emploi        2023-08  test  0.2   
45076     100      NaN  appli pole emploi        2023-08  test  0.2   
45074     100      NaN  appli pole emploi        2023-08  test  0.2   

       Anzahl_Vorkommen  rerank_score  
45072                 5     28.277011  
45073                 9 

In [17]:
#---Evaluation Vorbereitung---

# 1. Run-DF: qid, docno, rerank_score → score
Final_df_for_evaluation = (
    Final_df[['qid', 'docno', 'rerank_score']]
    .rename(columns={'rerank_score': 'score'})
    .copy()
)
Final_df_for_evaluation['qid']   = Final_df_for_evaluation['qid'].astype(str)
Final_df_for_evaluation['docno'] = Final_df_for_evaluation['docno'].astype(str)
Final_df_for_evaluation['score'] = Final_df_for_evaluation['score'].astype(float)

# 2. Qrels-DF: doc_id → docno, quer_id → qid  (Spalte 'label' bleibt erhalten)
df_qrel = df_qrel.rename(columns={'doc_id': 'docno', 'quer_id': 'qid'})
df_qrel['docno'] = df_qrel['docno'].astype(str).str.replace('doc', '', regex=False)
df_qrel['qid']   = df_qrel['qid'].astype(str)
# keine Ausschmückung von 'label' nötig, solange sie schon int ist

# 3. Kontrolle
print("Head of Final_df_for_evaluation:")
print(Final_df_for_evaluation)

print("\nHead of df_qrel:")
print(df_qrel)


Head of Final_df_for_evaluation:
        qid    docno      score
45072   100  1694624  28.277011
45073   100  2971472  26.165474
45075   100  2945156  26.165474
45076   100  3071601  26.165474
45074   100   143632  26.157930
...     ...      ...        ...
506470  996   960455   8.297803
506473  996   377417   8.296544
506474  996   401194   8.296544
506481  996   394087   8.295952
506484  996   546050   8.295305

[12583465 rows x 3 columns]

Head of df_qrel:
Empty DataFrame
Columns: [relevance, qid, docno]
Index: []


In [18]:
import pyterrier as pt

# 1. (Java läuft schon – kein erneutes pt.init())

# 2. Spalten in df_qrel umbenennen und typisieren
df_qrel = df_qrel.rename(columns={
    'quer_id':   'qid',
    'doc_id':    'docno',
    'relevance': 'label'
})

# Achte darauf, dass label nur ganze Zahlen enthält!
# Falls du Strings hast, etwa '1.0' oder '2.0', wandle sie vorher um:
df_qrel['label'] = df_qrel['label'].astype(float).astype(int)

df_qrel['qid']   = df_qrel['qid'].astype(str)
df_qrel['docno'] = df_qrel['docno'].astype(str)

# 3. Dein Run-DF vorbereiten
Final_df_for_evaluation = Final_df[['qid', 'docno', 'rerank_score']].rename(
    columns={'rerank_score': 'score'}
).copy()
Final_df_for_evaluation['qid']   = Final_df_for_evaluation['qid'].astype(str)
Final_df_for_evaluation['docno'] = Final_df_for_evaluation['docno'].astype(str)
Final_df_for_evaluation['score'] = Final_df_for_evaluation['score'].astype(float)

# 4. Evaluation durchführen
results = pt.Evaluate(
    res   = Final_df_for_evaluation,
    qrels = df_qrel,
    metrics = ["map", "ndcg_cut_10", "recall_100"],
  
 
)



# 5. Ausgabe
print("Evaluationsergebnisse (Durchschnitt über alle Queries):")
for metric, value in results.items():
    print(f"{metric}: {value:.4f}")



Evaluationsergebnisse (Durchschnitt über alle Queries):
map: nan
ndcg_cut_10: nan
recall_100: nan


In [19]:
run_name = 'tag'

# 1) Ranking pro Query-ID (qid) berechnen
Final_df['rank'] = (
    Final_df
      .sort_values(['qid', 'rerank_score'], ascending=[True, False])
      .groupby('qid')
      .cumcount() + 1
)

# 2) Nur die benötigten Spalten auswählen und in richtiger Reihenfolge
run_df = Final_df[['qid', 'docno', 'rank', 'rerank_score']].copy()
run_df['Q0']      = 'Q0'
run_df['run_name']= run_name

# Optional: Spalte für die TREC-Ausgabe wirklich 'query' nennen
run_df = run_df.rename(columns={'qid':'query'})

run_df = run_df[['query', 'Q0', 'docno', 'rank', 'rerank_score', 'run_name']]

# Hier cap auf Top 1000
max_rank = 1000
run_df = run_df[run_df['rank'] <= max_rank]

# 3) Als Textdatei exportieren – keine Header, kein Index
run_df.to_csv('trec_run_2023-08.run', sep=' ', header=False, index=False)

print(f"Run-Datei geschrieben als trec_run_file.run mit Top {max_rank}-Ergebnissen je Query")

Run-Datei geschrieben als trec_run_file.run mit Top 1000-Ergebnissen je Query
